In [1]:
import cv2
import numpy as np
from poly_point_isect import isect_segments

In [2]:
img = cv2.imread('imgs/cntrs.jpg')
img = cv2.resize(img, (0,0), fx=0.2, fy=0.2, interpolation=cv2.INTER_NEAREST)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

### Find corners

<p> Use the opencv function to find inner corners.
    
    note: uncomment j to see the result </p>

In [3]:
brd_image = np.copy(img)
ret, c = cv2.findChessboardCorners(img, (7,7), flags=cv2.CALIB_CB_ADAPTIVE_THRESH)
# j = cv2.drawChessboardCorners(brd_image, (7,7), c, ret)
corners = np.reshape(c, (7, 7, 2))

<p> Extrapolate the conners to find the outer squares <p>

In [4]:
extrap = np.zeros_like(corners, shape=(7, 9, 2))

for row in range(7):
    pre = corners[row][0] + corners[row][0] - corners[row][1]
    pro = corners[row][6] + corners[row][6] - corners[row][5]
    temp = np.insert(corners[row], 0, pre, axis=0)
    extrap[row] = np.insert(temp, 8, pro, axis=0)
    
pre = extrap[0] + extrap[0] - extrap[1]
pro = extrap[6] + extrap[6] - extrap[5]
temp = np.reshape(np.insert(extrap, 0, pre, axis=0), (8, 9, 2))
squares = np.reshape(np.insert(temp, 8, pro, axis=0), (9, 9, 2))

In [5]:
for row in squares:
    for x, y in row:
        cv2.circle(brd_image, (int(x), int(y)), 2, (0, 255, 0), 4)

### Find Counters 

In [8]:
for i in range(8):
    for j in range(8):
        # Grab the colour in the center of each square
        x, y = (squares[i][j] + squares[i+1][j] + squares[i][j+1] + squares[i+1][j+1]) / 4
        x = int(x)
        y = int(y)
        
        # Check red component to see if a counter is present
        if img[y, x][2] > 205:
            cv2.circle(brd_image, (x, y), 20, (255, 0, 0), 2)

# open image

In [7]:
cv2.imshow('test', brd_image)
cv2.waitKey(0)
cv2.destroyAllWindows()